# IEEE14 Powerflow

## Run simulation

### Import Libraries

In [ ]:
#import sys, os

#notebook_dir = os.path.abspath('')
#dpsim_root_dir = os.path.join(notebook_dir, "../../..")

#sys.path.insert(0, os.path.join(dpsim_root_dir, 'python/src/dpsim'))
#sys.path.insert(0, os.path.join(dpsim_root_dir, 'build'))

import dpsimpy
import matpower

from villas.dataprocessing.readtools import *
from villas.dataprocessing.timeseries import *
import urllib.request
import matplotlib.pyplot as plt

### Simulation parameters

In [ ]:
# simulation files
path_static_file = './IEEE14/ieee14.mat'

### 1. Powerflow for initialization

In [ ]:
sim_name_pf = 'IEEE14_PF'
dpsimpy.Logger.set_log_dir('logs/' + sim_name_pf)

# read and create dpsim topology
mpc_reader = matpower.Reader(mpc_file_path=path_static_file, mpc_name='ieee14')
mpc_reader.load_mpc(domain=matpower.Domain.PF)
system_pf = mpc_reader.system

# log results
logger = dpsimpy.Logger(sim_name_pf)
for node in system_pf.nodes:
    logger.log_attribute(node.name()+'.V', 'v', node)
    logger.log_attribute(node.name()+'.S', 's', node)

# Parametrize and run simulation
sim_pf = dpsimpy.Simulation(sim_name_pf, dpsimpy.LogLevel.info)
sim_pf.set_system(system_pf)
sim_pf.set_time_step(0.1)
sim_pf.set_final_time(0.1)
sim_pf.set_domain(dpsimpy.Domain.SP)
sim_pf.set_solver(dpsimpy.Solver.NRP)
sim_pf.do_init_from_nodes_and_terminals(False)
sim_pf.set_solver_component_behaviour(dpsimpy.SolverBehaviour.Initialization)
sim_pf.add_logger(logger)
sim_pf.run()

### Read pf results

In [ ]:
dpsim_result_file = 'logs/' + sim_name_pf + '/' + sim_name_pf + '.csv'
ts_dpsim_pf = read_timeseries_csv(dpsim_result_file)

#### DPsim results

In [ ]:
dpsim_results = pd.DataFrame(columns=['Bus', 'Vm [pu]', 'Va [°]', 'P [MW]', 'Q [MVAr]'])
base_power = 100 #mw
for i in range(len(system_pf.nodes)):
    node_name = system_pf.nodes[i].name() #ex. N5
    node_number = node_name.replace('N', '')
    node_baseV = mpc_reader.mpc_bus_data.loc[mpc_reader.mpc_bus_data['bus_i'] == int(node_number), 'baseKV'].iloc[0] * 1e3
    w_mw = 1e-6
    dpsim_results.loc[i] = ([node_name] + [round(np.absolute(ts_dpsim_pf[node_name + '.V'].values[-1]) / node_baseV, 5)]
        + [round(np.angle(ts_dpsim_pf[node_name + '.V'].values[-1])*180/np.pi, 5)] 
        + [round(w_mw * np.real(ts_dpsim_pf[node_name + '.S'].values[-1]), 5)] 
        + [round(w_mw * np.imag(ts_dpsim_pf[node_name + '.S'].values[-1]), 5)])

dpsim_results

### Validate Results against PSAT

In [ ]:
### PSAT Results
voltage_mag = np.array([1.06, 1.045, 1.01, 1.0186, 1.0203, 1.07, 1.062, 1.09, 1.0564, 1.0513, 1.0571, 1.0552, 1.0504, 1.0358])
voltage_angle = np.array([0, -0.08693, -0.22197, -0.18019, -0.15328, -0.24823, -0.23332, -0.23332, -0.26087, -0.26362, -0.25823, -0.26315, -0.26457, -0.27993])
p_loads = np.array([0, 0.217, 0.942, 0.478, 0.076, 0.112, 0, 0, 0.295, 0.09, 0.035, 0.061, 0.135, 0.149])
q_loads = np.array([0,  0.127, 0.19, -0.039, 0.016, 0.075, 0, 0, -0.04601, 0.058, 0.018, 0.016, 0.058, 0.05])
p_gen = np.array([2.3239, 0.4, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])
q_gen = np.array([-0.16889, 0.42396, 0.23394, 0, 0, 0.1224, 0, 0.17357, 0, 0, 0, 0, 0, 0])
busses = ["N1", "N2", "N3", "N4", "N5", "N6", "N7", "N8", "N9", "N10", "N11", "N12", "N13", "N14"]
d = {"Bus": busses, "V_mag(pu)": voltage_mag, "V_angle(deg)": voltage_angle, "P(pu)": -p_loads+p_gen, "Q(pu)": -q_loads+q_gen}
psat_results = pd.DataFrame(d)

psat_results

### Matpower results

In [ ]:
matpower_pf_results = mpc_reader.get_pf_results()
matpower_pf_results

### Difference

In [ ]:
difference_vmag = matpower_pf_results['Vm [pu]'] - dpsim_results['Vm [pu]']
difference_vangle = matpower_pf_results['Va [°]'] - dpsim_results['Va [°]']

#base power = 100MW
difference_p = (matpower_pf_results['P [MW]'] - dpsim_results['P [MW]'])
difference_q = (matpower_pf_results['Q [MVAr]'] - dpsim_results['Q [MVAr]'])
difference = {"Bus": matpower_pf_results['Bus'], "V_mag [pu]": difference_vmag, "Va [°]": difference_vangle, "P [MW]": difference_p, "Q [MVAr]": difference_q}
difference = pd.DataFrame(difference)

difference

### Assertion

In [ ]:
tolerance = 5e-5

print(np.max(np.absolute(difference_p)))
assert(np.all(np.absolute(difference_p)<tolerance))

print(np.max(np.absolute(difference_q)))
assert(np.all(np.absolute(difference_q)<tolerance))

print(np.max(np.absolute(difference_vmag)))
assert(np.all(np.absolute(difference_vmag)<tolerance))

print(np.max(np.absolute(difference_vangle)))
assert(np.all(np.absolute(difference_vangle)<tolerance))